# Reinforcement Learning Agent Simulation for TicTacToe

In [1]:
import random
import math
from matplotlib import pyplot as plt
from statistics import mean 

In [2]:
class GameTree():
    alpha = 0.1
    curr_state = None
    root=""
    
    def altsym(sym):
        if sym == 'X':
            return 'O'
        else:
            return 'X'
    
    def check_win(state, sym):
        winstr = sym+sym+sym
        for i in range(0,9,3):
            if state[i]+state[i+1]+state[i+2] == winstr:
                return True
        for i in range(0,3):
            if state[i]+state[i+3]+state[i+6] == winstr:
                return True
        if (state[0]+state[4]+state[8] == winstr) or (state[2]+state[4]+state[6] == winstr):
            return True
        return False
    
    ################################################
    
    def __init__(self, sym="", state=["_" for i in range(9)], value = 0.5, children=None, parent=None):
        self.sym = sym
        self.state = state
        self.value = value
        self.parent = parent
        self.children = []
        if children is not None:
            for child in children:                
                self.add_child(child)
        
    def add_child(self, node):
        assert isinstance(node, GameTree)
        #for child in self.children:
        #    if child.state == node.state:
        #        return
        node.parent = self
        self.children.append(node)
        
    def backup_val(self):
        if self.parent != None and self.parent.parent != None:
            self.parent.parent.value = self.parent.parent.value + self.alpha * (self.value - self.parent.parent.value)
            if (not TicTacToe.train_mode): print("Value Updated!")
        
    def update_enemy_state(self, state):
        for child in self.curr_state.children:          #simulate opponent move
            if child.state == state:
                self.curr_state = child
                break
                
    def update_state(self,state):
        i = random.randrange(len(self.curr_state.children))      
        greedy_state = self.curr_state.children[i]
        
        gen = random.random()
        if gen >= 0.1:
            for child in self.curr_state.children:     #select epsilon greedy move 
                if child.value > greedy_state.value:
                    greedy_state = child
            self.curr_state = greedy_state
            self.curr_state.backup_val()
        else:
            self.curr_state = greedy_state                    
        
    
    def gen_states(self, pcsym, plsym):
        if GameTree.check_win(self.state, pcsym):
            self.value = 1
            return
        elif GameTree.check_win(self.state, plsym):
            self.value = 0
            return
        if "_" in self.state:
            for i in range(9):
                if self.state[i] == "_":
                    cstate = self.state[:]
                    cstate[i] = self.sym
                    self.add_child(GameTree(GameTree.altsym(self.sym), cstate))
        else:
            self.value = 0


In [3]:
class TicTacToe:
    initflag = 1
    train_mode = 0
    def player_input(state, sym):
        chk = 1
        while chk:
            chk=0
            plmove = input("Enter your move (1-9): ")
            if plmove not in [str(i) for i in range(10)]:
                chk = 1
                print("Beep Bop! Invalid Input!")
                continue
            plmove = int(plmove)
            if not TicTacToe.check_move(plmove-1, state):
                chk = 1
                print("Beep Bop! Invalid Input!")
                continue
        state[plmove - 1] = sym
        return state
        
    def pc_input_rand(state, sym):
        chk = 1
        if(not TicTacToe.train_mode):
            print("'{}' PCs Turn!".format(sym))
        while chk:
            chk=0
            pcmove = random.randrange(9)
            if not TicTacToe.check_move(pcmove, state):
                chk = 1                
        state[pcmove] = sym
        return state
    
    def pc_input_expert(state, sym):
        chk = 1
        if(not TicTacToe.train_mode):
            print("'{}' PCs Turn!".format(sym))        
        thinkstate = state[:]
        osym = plsym if sym==pcsym else pcsym
        
        #Case 0: Init
        if ("X" not in state) and ("O" not in state):
            state[random.choice([0,2,4,6,8])] = sym
            if(not TicTacToe.train_mode):
                print("Played Move 0! (Starting Move)")
            return state
        
        #Case 1: winning move
        for i in range(9):
            if TicTacToe.check_move(i, thinkstate):
                thinkstate[i] = sym
                if TicTacToe.check_win(thinkstate, sym):
                    state[i] = sym
                    if(not TicTacToe.train_mode):
                        print("Played Move 1! (Winning Move)")
                    return state
                thinkstate[i] = "_"
                
        #Case 2: block        
        for i in range(9):
            if TicTacToe.check_move(i, thinkstate):
                thinkstate[i] = osym
                if TicTacToe.check_win(thinkstate, osym):
                    state[i] = sym
                    if(not TicTacToe.train_mode):
                        print("Played Move 2! (Block)")
                    return state
                thinkstate[i] = "_"
                
        #Case 3: fork        
        for i in range(9):
            if TicTacToe.check_move(i, thinkstate):
                thinkstate[i] = sym
                winc=0
                for j in range(9):
                    if TicTacToe.check_move(j, thinkstate):
                        thinkstate[j] = sym                        
                        if TicTacToe.check_win(thinkstate, sym):
                            winc+=1
                        if winc >= 2:
                            state[i] = sym
                            if(not TicTacToe.train_mode):
                                print("Played Move 3! (Fork)")
                            return state                    
                        thinkstate[j] = "_"
                thinkstate[i] = "_"
        
        #Case 4: block fork
        for i in range(9):
            if TicTacToe.check_move(i, thinkstate):
                thinkstate[i] = osym
                winc=0
                for j in range(9):
                    if TicTacToe.check_move(j, thinkstate):
                        thinkstate[j] = osym                        
                        if TicTacToe.check_win(thinkstate, osym):
                            winc+=1                                            
                        thinkstate[j] = "_"
                        
                if winc >= 2:
                    for j in range(9):
                        if TicTacToe.check_move(j, thinkstate):
                            thinkstate[j] = sym
                            winc2=0
                            for k in range(9):
                                if TicTacToe.check_move(k, thinkstate):
                                    thinkstate[k] = sym                        
                                    if TicTacToe.check_win(thinkstate, sym):                                        
                                        state[j] = sym
                                        if(not TicTacToe.train_mode):
                                            print("Played Move 4! (counter Fork)")
                                        return state                    
                                    thinkstate[k] = "_"
                            thinkstate[j] = "_"   
                    state[i] = sym
                    if(not TicTacToe.train_mode):
                        print("Played Move 4! (block fork)")
                    return state                    
                thinkstate[i] = "_"
        
        #Case 5: centre
        if state[4]=="_":
            state[4] = sym
            if(not TicTacToe.train_mode):
                print("Played Move 5! (centre)")
            return state
        
        #Case 6: opp corner 
        movs = [0,2,6,8]
        for i in range(4):
            if state[movs[3-i]]=="_":
                if state[movs[i]]==osym:
                    state[movs[3-i]]=sym
                    if(not TicTacToe.train_mode):
                        print("Played Move 6! (opp corner)")
                    return state
        
        #Case 7: empty corner
        for i in random.sample([0,2,6,8],4):
            if state[i]=="_":
                state[i] = sym
                if(not TicTacToe.train_mode):
                    print("Played Move 7! (empty corner)")
                return state
        
        #Case 8: empty side
        for i in random.sample([1,3,5,7],4):
            if state[i]=="_":
                state[i] = sym
                if(not TicTacToe.train_mode):
                    print("Played Move 8! (empty side)")
                return state
        
        print("Confused!")
        return pc_input_rand(state, sym)
    
    def pc_input_rl(state, sym, pctree):
        if(not TicTacToe.train_mode):
            print("'{}' PCs Turn!".format(sym))
        if TicTacToe.initflag:
            pctree.curr_state = pctree
            TicTacToe.initflag = 0
        if sym == 'X':
            GameTree.root = 'X'            
        else:
            GameTree.root = 'O'                    
        
        if pctree.curr_state.state == state:
            #print(state, pctree.curr_state.state)
            pctree.update_state(state)
            #print(state, pctree.curr_state.state)
        else:
            #print(state, pctree.curr_state.state)
            pctree.update_enemy_state(state)
            #print(state, pctree.curr_state.state)
            assert (pctree.curr_state.state == state)
            pctree.update_state(state)
            #print(state, pctree.curr_state.state)
        
        state = pctree.curr_state.state[:]
        return state
    
    def display_board(state):
        for i in range(0,9,3):
            print("{}|{}|{}".format(state[9-i-3],state[9-i-2],state[9-i-1]))
        print("  ")
    
    def check_status(state, disp=1):
        if TicTacToe.check_win(state, plsym):
            if (disp): print("You Won!")
            return 0
        elif TicTacToe.check_win(state, pcsym):
            if (disp): print("You Lost!")
            return 0
        elif "_" not in state:
            if (disp): print("Draw! Game Over!")
            return 0
        else:
            return 1    
    
    def check_move(move, state):
        if state[move] == '_':
            return True
        else:
            return False
        
    def check_win(state, sym):
        winstr = sym+sym+sym
        for i in range(0,9,3):
            if state[i]+state[i+1]+state[i+2] == winstr:
                return True
        for i in range(0,3):
            if state[i]+state[i+3]+state[i+6] == winstr:
                return True
        if (state[0]+state[4]+state[8] == winstr) or (state[2]+state[4]+state[6] == winstr):
            return True
        return False
    

## User Play:- Player vs RL Agent

In [4]:
print("Let's Play TicTacToe!")
chk = 1
while chk:
    chk=0
    sym = input("Enter O or X: ")
    if sym not in ('o','O','x','X'):
        chk = 1
        print("Beep Bop! Invalid Input!")
        
plsym = 'X' if (sym=='X' or sym=='x') else 'O'
pcsym = 'O' if (sym=='X' or sym=='x') else 'X'

print("You chose {}. Time for a toss!".format(plsym))
input("Press anykey to toss!")
toss = random.random()
turn = 0
if toss<0.5:
    print("You won the toss! So start the game first!")
    turn = 1
    if (plsym=='X'):
        pctree = GameTree('X')         
        pctree.root = 'O'        
    else:
        pctree = GameTree('O')        
        pctree.root = 'X'        
else :
    print("PC won the toss!")
    if (pcsym=='X'):
        pctree = GameTree('X')         
        pctree.root = 'X'
    else:
        pctree = GameTree('O')
        pctree.root = 'O'        
sav_turn = turn

print("Loading RL Agent!.....")
stack = [pctree]
pctree.curr_state = pctree
while len(stack) != 0:
    curtree = stack.pop()
    curtree.gen_states(pcsym, plsym)
    for child in curtree.children[::-1]:
        stack.append(child)    
print("RL Agent is Ready!")

Let's Play TicTacToe!
Enter O or X: x
You chose X. Time for a toss!
Press anykey to toss!
You won the toss! So start the game first!
Loading RL Agent!.....
RL Agent is Ready!


In [5]:
from collections import deque

queue = deque([pctree])
for _ in range(100):
    curtree = queue.popleft()
    print(curtree.state)
    for child in curtree.children:
        queue.append(child)

['_', '_', '_', '_', '_', '_', '_', '_', '_']
['X', '_', '_', '_', '_', '_', '_', '_', '_']
['_', 'X', '_', '_', '_', '_', '_', '_', '_']
['_', '_', 'X', '_', '_', '_', '_', '_', '_']
['_', '_', '_', 'X', '_', '_', '_', '_', '_']
['_', '_', '_', '_', 'X', '_', '_', '_', '_']
['_', '_', '_', '_', '_', 'X', '_', '_', '_']
['_', '_', '_', '_', '_', '_', 'X', '_', '_']
['_', '_', '_', '_', '_', '_', '_', 'X', '_']
['_', '_', '_', '_', '_', '_', '_', '_', 'X']
['X', 'O', '_', '_', '_', '_', '_', '_', '_']
['X', '_', 'O', '_', '_', '_', '_', '_', '_']
['X', '_', '_', 'O', '_', '_', '_', '_', '_']
['X', '_', '_', '_', 'O', '_', '_', '_', '_']
['X', '_', '_', '_', '_', 'O', '_', '_', '_']
['X', '_', '_', '_', '_', '_', 'O', '_', '_']
['X', '_', '_', '_', '_', '_', '_', 'O', '_']
['X', '_', '_', '_', '_', '_', '_', '_', 'O']
['O', 'X', '_', '_', '_', '_', '_', '_', '_']
['_', 'X', 'O', '_', '_', '_', '_', '_', '_']
['_', 'X', '_', 'O', '_', '_', '_', '_', '_']
['_', 'X', '_', '_', 'O', '_', '_'

In [6]:
TicTacToe.initflag = 1
gameon = 1
turn = sav_turn
state = ["_" for i in range(9)]
while gameon:
    if turn:
        TicTacToe.display_board(state)
        #print(state, pctree.curr_state.state, "in pl game loop")
        state = TicTacToe.player_input(state, plsym) 
        #print(state, pctree.curr_state.state, "in pl game loop")
        gameon = TicTacToe.check_status(state)
        turn = 0
    else:
        TicTacToe.display_board(state)
        #print(state, pctree.curr_state.state, "in pc game loop")
        state = TicTacToe.pc_input_rl(state, pcsym, pctree)
        #print(state, pctree.curr_state.state, "in pc game loop")
        gameon = TicTacToe.check_status(state)
        turn = 1
        
TicTacToe.display_board(state)  
if TicTacToe.check_win(state, pcsym) or ("_" not in state and not sav_turn):
    pctree.curr_state.backup_val()
elif TicTacToe.check_win(state, plsym) or ("_" not in state and sav_turn):
    pctree.update_enemy_state(state)
    assert (pctree.curr_state.state == state)
    pctree.curr_state.parent.value = pctree.curr_state.value
    pctree.curr_state = pctree.curr_state.parent
    pctree.curr_state.backup_val()
    pctree.update_enemy_state(state)
            

_|_|_
_|_|_
_|_|_
  
Enter your move (1-9): 1
_|_|_
_|_|_
X|_|_
  
'O' PCs Turn!
Value Updated!
_|_|O
_|_|_
X|_|_
  
Enter your move (1-9): 5
_|_|O
_|X|_
X|_|_
  
'O' PCs Turn!
Value Updated!
_|_|O
_|X|_
X|O|_
  
Enter your move (1-9): 7
X|_|O
_|X|_
X|O|_
  
'O' PCs Turn!
Value Updated!
X|_|O
_|X|_
X|O|O
  
Enter your move (1-9): 4
You Won!
X|_|O
X|X|_
X|O|O
  
Value Updated!


In [22]:
statevar = pctree.curr_state
while statevar!=None:     
    print(statevar.state, statevar.value)
    statevar = statevar.parent

['X', 'O', 'X', 'X', 'X', 'O', 'O', 'X', 'O'] 0
['_', 'O', 'X', 'X', 'X', 'O', 'O', 'X', 'O'] 0
['_', 'O', 'X', 'X', 'X', '_', 'O', 'X', 'O'] 0.5
['_', 'O', 'X', '_', 'X', '_', 'O', 'X', 'O'] 2.8297056892220874e-06
['_', '_', 'X', '_', 'X', '_', 'O', 'X', 'O'] 0.5
['_', '_', 'X', '_', 'X', '_', 'O', '_', 'O'] 0.831593266406187
['_', '_', 'X', '_', 'X', '_', '_', '_', 'O'] 0.5
['_', '_', '_', '_', 'X', '_', '_', '_', 'O'] 0.7783027655892014
['_', '_', '_', '_', 'X', '_', '_', '_', '_'] 0.5
['_', '_', '_', '_', '_', '_', '_', '_', '_'] 0.9088654016720712


### Training RL Agent

In [20]:
print("Training Agent!")
for i in range(10000):
    TicTacToe.initflag = 1
    TicTacToe.train_mode = 1
    gameon = 1
    turn = sav_turn
    state = ["_" for i in range(9)]
    while gameon:
        if turn:
            #TicTacToe.display_board(state)
            #print(state, pctree.curr_state.state, "in pl game loop")
            state = TicTacToe.pc_input_rand(state, plsym) 
            #print(state, pctree.curr_state.state, "in pl game loop")
            gameon = TicTacToe.check_status(state, 0)
            turn = 0
        else:
            #TicTacToe.display_board(state)
            #print(state, pctree.curr_state.state, "in pc game loop")
            state = TicTacToe.pc_input_rl(state, pcsym, pctree)
            #print(state, pctree.curr_state.state, "in pc game loop")
            gameon = TicTacToe.check_status(state, 0)
            turn = 1
            
    #TicTacToe.display_board(state)  
    if TicTacToe.check_win(state, pcsym) or ("_" not in state and not sav_turn):
        pctree.curr_state.backup_val()
    elif TicTacToe.check_win(state, plsym) or ("_" not in state and sav_turn):
        pctree.update_enemy_state(state)
        assert (pctree.curr_state.state == state)
        pctree.curr_state.parent.value = pctree.curr_state.value
        pctree.curr_state = pctree.curr_state.parent
        pctree.curr_state.backup_val()
        pctree.update_enemy_state(state)
        
    for i in range(10000):
    TicTacToe.initflag = 1
    TicTacToe.train_mode = 1
    gameon = 1
    turn = sav_turn
    state = ["_" for i in range(9)]
    while gameon:
        if turn:
            #TicTacToe.display_board(state)
            #print(state, pctree.curr_state.state, "in pl game loop")
            state = TicTacToe.pc_input_expert(state, plsym) 
            #print(state, pctree.curr_state.state, "in pl game loop")
            gameon = TicTacToe.check_status(state, 0)
            turn = 0
        else:
            #TicTacToe.display_board(state)
            #print(state, pctree.curr_state.state, "in pc game loop")
            state = TicTacToe.pc_input_rl(state, pcsym, pctree)
            #print(state, pctree.curr_state.state, "in pc game loop")
            gameon = TicTacToe.check_status(state, 0)
            turn = 1
            
    #TicTacToe.display_board(state)  
    if TicTacToe.check_win(state, pcsym) or ("_" not in state and not sav_turn):
        pctree.curr_state.backup_val()
    elif TicTacToe.check_win(state, plsym) or ("_" not in state and sav_turn):
        pctree.update_enemy_state(state)
        assert (pctree.curr_state.state == state)
        pctree.curr_state.parent.value = pctree.curr_state.value
        pctree.curr_state = pctree.curr_state.parent
        pctree.curr_state.backup_val()
        pctree.update_enemy_state(state)
    
print("Training Done!")

Training Agent!
Training Done!


In [21]:
TicTacToe.initflag = 1
TicTacToe.train_mode = 0
gameon = 1
turn = sav_turn
state = ["_" for i in range(9)]
while gameon:
    if turn:
        TicTacToe.display_board(state)
        #print(state, pctree.curr_state.state, "in pl game loop")
        state = TicTacToe.player_input(state, plsym) 
        #print(state, pctree.curr_state.state, "in pl game loop")
        gameon = TicTacToe.check_status(state)
        turn = 0
    else:
        TicTacToe.display_board(state)
        #print(state, pctree.curr_state.state, "in pc game loop")
        state = TicTacToe.pc_input_rl(state, pcsym, pctree)
        #print(state, pctree.curr_state.state, "in pc game loop")
        gameon = TicTacToe.check_status(state)
        turn = 1
        
TicTacToe.display_board(state)  
if TicTacToe.check_win(state, pcsym) or ("_" not in state and not sav_turn):
    pctree.curr_state.backup_val()
elif TicTacToe.check_win(state, plsym) or ("_" not in state and sav_turn):
    pctree.update_enemy_state(state)
    assert (pctree.curr_state.state == state)
    pctree.curr_state.parent.value = pctree.curr_state.value
    pctree.curr_state = pctree.curr_state.parent
    pctree.curr_state.backup_val()
    pctree.update_enemy_state(state)
    
    

_|_|_
_|_|_
_|_|_
  
Enter your move (1-9): 5
_|_|_
_|X|_
_|_|_
  
'O' PCs Turn!
Value Updated!
_|_|O
_|X|_
_|_|_
  
Enter your move (1-9): 3
_|_|O
_|X|_
_|_|X
  
'O' PCs Turn!
Value Updated!
O|_|O
_|X|_
_|_|X
  
Enter your move (1-9): 8
O|X|O
_|X|_
_|_|X
  
'O' PCs Turn!
Value Updated!
O|X|O
_|X|_
_|O|X
  
Enter your move (1-9): 4
O|X|O
X|X|_
_|O|X
  
'O' PCs Turn!
Value Updated!
O|X|O
X|X|O
_|O|X
  
Enter your move (1-9): 1
Draw! Game Over!
O|X|O
X|X|O
X|O|X
  
Value Updated!


## User Play:- Player vs Random

In [ ]:
print("Let's Play TicTacToe!")
chk = 1
while chk:
    chk=0
    sym = input("Enter O or X: ")
    if sym not in ('o','O','x','X'):
        chk = 1
        print("Beep Bop! Invalid Input!")
        
plsym = 'X' if (sym=='X' or sym=='x') else 'O'
pcsym = 'O' if (sym=='X' or sym=='x') else 'X'

print("You chose {}. Time for a toss!".format(plsym))
input("Press anykey to toss!")
toss = random.random()
turn = 0
if toss<0.5:
    print("You won the toss! So start the game first!")
    turn = 1
else :
    print("PC won the toss!")

gameon = 1
state = ["_" for i in range(9)]
while gameon:
    if turn:
        TicTacToe.display_board(state)
        state = TicTacToe.player_input(state, plsym)        
        gameon = TicTacToe.check_status(state)
        turn = 0
    else:
        TicTacToe.display_board(state)
        state = TicTacToe.pc_input_rand(state, pcsym)        
        gameon = TicTacToe.check_status(state)
        turn = 1
        
TicTacToe.display_board(state)    

## User Play:- Player vs Strategic

In [ ]:
print("Let's Play TicTacToe!")
chk = 1
while chk:
    chk=0
    sym = input("Enter O or X: ")
    if sym not in ('o','O','x','X'):
        chk = 1
        print("Beep Bop! Invalid Input!")
        
plsym = 'X' if (sym=='X' or sym=='x') else 'O'
pcsym = 'O' if (sym=='X' or sym=='x') else 'X'

print("You chose {}. Time for a toss!".format(plsym))
input("Press anykey to toss!")
toss = random.random()
turn = 0
if toss<0.5:
    print("You won the toss! So start the game first!")
    turn = 1
else :
    print("PC won the toss!")

gameon = 1
state = ["_" for i in range(9)]
while gameon:
    if turn:
        TicTacToe.display_board(state)
        state = TicTacToe.player_input(state, plsym)        
        gameon = TicTacToe.check_status(state)
        turn = 0
    else:
        TicTacToe.display_board(state)
        state = TicTacToe.pc_input_expert(state, pcsym)        
        gameon = TicTacToe.check_status(state)
        turn = 1
        
TicTacToe.display_board(state)    

## Self Play:- Strategic vs Random

In [ ]:
print("Let's Play TicTacToe!")
chk = 1
while chk:
    chk=0
    sym = input("Enter O or X: ")
    if sym not in ('o','O','x','X'):
        chk = 1
        print("Beep Bop! Invalid Input!")
        
plsym = 'X' if (sym=='X' or sym=='x') else 'O'
pcsym = 'O' if (sym=='X' or sym=='x') else 'X'

print("You chose {}. Time for a toss!".format(plsym))
input("Press anykey to toss!")
toss = random.random()
turn = 0
if toss<0.5:
    print("Your Agent '{} PC' won the toss! So start the game first!".format(plsym))
    turn = 1
else :
    print("'{}' PC won the toss!".format(pcsym))

gameon = 1
state = ["_" for i in range(9)]
while gameon:
    if turn:
        TicTacToe.display_board(state)
        state = TicTacToe.pc_input_expert(state, plsym)        
        gameon = TicTacToe.check_status(state)
        turn = 0
    else:
        TicTacToe.display_board(state)
        state = TicTacToe.pc_input_rand(state, pcsym)        
        gameon = TicTacToe.check_status(state)
        turn = 1

TicTacToe.display_board(state)    

## Self Play:- Strategic Vs Strategic

In [ ]:
print("Let's Play TicTacToe!")
chk = 1
while chk:
    chk=0
    sym = input("Enter O or X: ")
    if sym not in ('o','O','x','X'):
        chk = 1
        print("Beep Bop! Invalid Input!")
        
plsym = 'X' if (sym=='X' or sym=='x') else 'O'
pcsym = 'O' if (sym=='X' or sym=='x') else 'X'

print("You chose {}. Time for a toss!".format(plsym))
input("Press anykey to toss!")
toss = random.random()
turn = 0
if toss<0.5:
    print("Your Agent '{} PC' won the toss! So start the game first!".format(plsym))
    turn = 1
else :
    print("'{}' PC won the toss!".format(pcsym))

gameon = 1
state = ["_" for i in range(9)]
while gameon:
    if turn:
        TicTacToe.display_board(state)
        state = TicTacToe.pc_input_expert(state, plsym)        
        gameon = TicTacToe.check_status(state)
        turn = 0
    else:
        TicTacToe.display_board(state)
        state = TicTacToe.pc_input_expert(state, pcsym)        
        gameon = TicTacToe.check_status(state)
        turn = 1

TicTacToe.display_board(state)    

In [ ]:
curtree = pctree
while True:
    print(curtree.state, curtree.value)
    if len(curtree.children) == 0:
        break
    i = random.randrange(len(curtree.children))
    curtree = curtree.children[i]